In [ ]:
!pip install imutils

In [3]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense, Input, Lambda, Input, Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D
import tensorflow as tf 
import tensorflow.keras as keras 

In [2]:
from imutils import paths 

train_image_files = list(paths.list_images('train'))
test_image_files = list(paths.list_images('test'))
val_image_files = list(paths.list_images('val'))

In [ ]:
from tensorflow import keras 
import tensorflow as tf 

# Image preprocessing utils
def parse_images(image_path): 
    """
        this function, will read the image from the directory, and resize it to the 224 image shape, which is predefined 
        image shape of Resnet pretrained model.
        Params:
            image_path(type: str): image filepath in the directory.
        Return(type: tf.Image)
            returns the preprocessed image.
    """
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[224, 224])

    return image

def create_tensorflow_dataset(data, batch_size):
    """
        this function, will create the tensorflow dataset from the images.
        Params:
            data(type: List): list of data, in our case list contains the file path.
            batch_size(type: Int): batch size for the generator.
        Return(type: tf.data.Dataset)
            returns tensorflow dataset object.
    """
    tensorflow_data = tf.data.Dataset.from_tensor_slices(data)
    tensorflow_data = (
    tensorflow_data
        .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .shuffle(1024)
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return tensorflow_data        

In [76]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

def save_loss_plot(fpath, history): 
    with plt.xkcd():
        plt.plot(history.history["loss"], label="train_loss")
        plt.plot(history.history["val_loss"], label="val_loss")

        plt.title("Training Loss and val loss")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss")
        plt.legend(loc="lower left")
    plt.savefig("plots/loss_linearevaluation_1.png")
    

def save_acc_plot(fpath, history): 
    with plt.xkcd():
        plt.plot(history.history["accuracy"], label="train_acc")
        plt.plot(history.history["val_accuracy"], label="val_acc")
        plt.title("Training and val Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Accuracy")
        plt.legend(loc="lower left")
    plt.savefig(fpath)

    
def create_dataset(images_files): 
    X = []
    y = []
    for image_file in images_files:
        try: 
            img = cv2.imread(image_file)
            img = cv2.resize(img, (224, 224))
            img = img / 255.0 

            label = image_file.split("/")[1].split("_")[0]

            X.append(img)
            y.append(label)
        
        except Exception as error:
            return error
    
    return np.array(X), np.array(y)

In [77]:
train_X, train_y = create_dataset(train_image_files)
train_X.shape

(2700, 224, 224, 3)

In [78]:
np.unique(train_y)

array(['airplane', 'car', 'elephant', 'lion', 'mouse', 'tiger cub'],
      dtype='<U9')

In [79]:
val_X, val_y = create_dataset(val_image_files)
val_X.shape

(292, 224, 224, 3)

In [80]:
test_X, test_y = create_dataset(test_image_files)
test_X.shape

(300, 224, 224, 3)

In [81]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
val_y = label_encoder.transform(val_y)
test_y = label_encoder.transform(test_y)

In [45]:
base_encoder_model = keras.models.load_model("simclr_model_1")

In [46]:
base_encoder_model =  keras.models.Sequential([
    base_encoder_model.layers[0],
    base_encoder_model.layers[1]
])

In [47]:
base_encoder_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [48]:
def create_reprsentational_features(model, num_iter, data): 
    features_reprsentation = model(data[:50])
    for i in range(int(num_iter)-1): 
        start_indx = i+1 * 50 
        end_indx = start_indx + 50 

        features_reprsentation = tf.concat([features_reprsentation, model(data[start_indx: end_indx])], axis=0)
        
    return features_reprsentation

In [49]:
num_iter = train_X.shape[0] / 50
train_features_reprsentation = create_reprsentational_features(base_encoder_model, num_iter, train_X)

In [50]:
train_features_reprsentation.shape

TensorShape([2700, 2048])

In [51]:
num_iter = test_X.shape[0] / 50
print(num_iter)
val_features_reprsentation = create_reprsentational_features(base_encoder_model, num_iter, test_X)

6.0


In [52]:
train_features_reprsentation = tf.concat([train_features_reprsentation, val_features_reprsentation], axis=0)
train_features_reprsentation.shape

TensorShape([3000, 2048])

In [53]:
num_iter = val_X.shape[0] / 50
test_features_reprsentation = create_reprsentational_features(base_encoder_model, num_iter, val_X)

In [54]:
train_features_reprsentation = train_features_reprsentation.numpy()
test_features_reprsentation = test_features_reprsentation.numpy()

In [84]:
from sklearn.decomposition import PCA

In [85]:
pca_transformer = PCA(n_components=1500)
pca_transformer.fit(train_features_reprsentation)

PCA(n_components=1500)

In [86]:
train_X = pca_transformer.transform(train_features_reprsentation)
train_y = np.concatenate([train_y, test_y], axis=0)
train_y.shape

(3000,)

In [87]:
test_X = pca_transformer.transform(test_features_reprsentation)
test_y = val_y[:250]

In [88]:
from sklearn.linear_model import LogisticRegression

In [89]:
lg = LogisticRegression(max_iter=10000)
lg.fit(train_X, train_y)

LogisticRegression(max_iter=10000)

In [90]:
pred_y = lg.predict(test_X)
pred_y

array([0, 3, 2, 3, 4, 5, 3, 0, 2, 5, 5, 2, 4, 1, 0, 1, 0, 3, 2, 4, 5, 3,
       2, 4, 1, 2, 5, 2, 1, 5, 1, 1, 0, 4, 3, 1, 0, 3, 4, 4, 2, 3, 5, 1,
       1, 2, 1, 5, 0, 3, 4, 3, 0, 4, 1, 2, 4, 2, 4, 4, 2, 4, 0, 4, 0, 4,
       1, 5, 0, 5, 0, 2, 4, 3, 3, 3, 3, 5, 5, 5, 4, 3, 1, 0, 5, 0, 3, 4,
       5, 0, 5, 2, 5, 1, 5, 2, 1, 1, 0, 5, 4, 0, 4, 1, 0, 4, 2, 4, 3, 1,
       4, 2, 0, 0, 4, 1, 4, 3, 5, 5, 0, 3, 0, 5, 1, 5, 0, 5, 5, 4, 3, 1,
       0, 5, 0, 4, 4, 5, 4, 5, 2, 5, 1, 5, 2, 1, 5, 0, 3, 2, 2, 3, 3, 4,
       4, 3, 4, 3, 2, 3, 2, 0, 0, 4, 4, 4, 1, 5, 2, 0, 4, 1, 5, 3, 3, 0,
       5, 5, 4, 3, 0, 3, 5, 5, 4, 4, 3, 0, 0, 2, 5, 1, 1, 0, 2, 5, 0, 3,
       1, 2, 1, 0, 5, 2, 3, 4, 1, 2, 3, 2, 0, 3, 4, 4, 1, 1, 3, 2, 0, 4,
       2, 5, 3, 5, 5, 5, 5, 4, 3, 5, 5, 5, 0, 3, 3, 3, 0, 2, 4, 3, 1, 4,
       0, 2, 5, 0, 1, 4, 4, 5])

In [91]:
train_y

array([2, 1, 0, ..., 0, 3, 0])

In [92]:
from sklearn.metrics import *

print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.21      0.26      0.24        34
           1       0.21      0.18      0.19        38
           2       0.24      0.22      0.23        37
           3       0.19      0.16      0.18        49
           4       0.17      0.17      0.17        47
           5       0.26      0.29      0.27        45

    accuracy                           0.21       250
   macro avg       0.21      0.21      0.21       250
weighted avg       0.21      0.21      0.21       250



In [93]:
train_y.shape

(3000,)